In [ ]:
import cv2 as cv
from cv2 import VideoCapture

In [ ]:
from box_extractor import BoxExtractor, find_boxes

extractor = BoxExtractor(find_boxes, slice_size=400, object_size=150)

cap = VideoCapture("worms.avi")

i = 0
while True:
    ret, image = cap.read()
    if ret == False:
        break

    if cv.waitKey(1) & 0xFF == ord("q"):
        break

    coords, rois = extractor.run(image)

    # Draw bboxes
    for box in coords:
        x, y, w, h = box
        cv.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 3)

    cv.imshow("BBOXES", image)

    i += 1

cap.release()
cv.destroyAllWindows()

In [ ]:
from view_controller import ViewController
from timing_context import TimingContext

cap = VideoCapture("worms.avi")

sim = ViewController(cap, color_conversion=cv.COLOR_BGR2GRAY)

while True:
    with TimingContext():
        sim.next_frame()

    sim.visualize_world()
    sim.move_position(5, 5)

    if cv.waitKey(1) & 0xFF == ord("q"):
        break

cv.destroyAllWindows()

In [2]:
import math

fps = 40
micro_view_time = 200
tracking_time = 50
moving_time = 25

frame_time = 1 / fps
micro_view_frames = math.ceil(micro_view_time / frame_time)
tracking_frames = math.ceil(tracking_time / frame_time)
moving_frames = math.ceil(moving_time / frame_time)

while True:
    # Here we assume that the worm is kinda centered on the screen
    sim.micro_view()  # Takes 'micro_frames' frames, can do other stuff meanwhile
    sim.next_frame(micro_view_frames - tracking_frames)

    cam_view = sim.camera_view()
    dx, dy = real_time_tracking(cam_view)  # Takes 'track_frames' frames, can't do anything meanwhile
    sim.next_frame(tracking_frames)  # micro and tracking finished

    sim.next_frame(moving_frames)  # moving takes 'move_frames' frames, can do other stuff meanwhile
    sim.move_position(dx, dy)

NameError: name 'sim' is not defined